# Pose Graph Relaxation Example

Example that constructs and solves a relative pose graph problem.

In [1]:
import numpy as np

from pylgmath import Transformation
from pysteam.problem import StaticNoiseModel, L2LossFunc, WeightedLeastSquareCostTerm, OptimizationProblem
from pysteam.solver import GaussNewtonSolver
from pysteam.state import TransformStateVar
from pysteam.evaluator import TransformErrorEval

In [2]:
# Options
num_poses = 100
T_k2_k1_vec = np.array([[-1.0, 0., 0., 0., 0., 0.01]]).T
measurements = [((i - 1, i, Transformation(xi_ab=T_k2_k1_vec))) for i in range(1, num_poses)]

# setup states
states = [Transformation() for _ in range(num_poses)]
state_vars = [TransformStateVar(state) for state in states]
state_vars[0].set_lock(True)  # lock first pose (otherwise entire solution is 'floating')

# turn measurements into cost terms
cost_terms = []
noise_model = StaticNoiseModel(np.eye(6))
loss_func = L2LossFunc()
for meas in measurements:
  T_k1_0 = state_vars[meas[0]]
  T_k2_0 = state_vars[meas[1]]
  T_k2_k1 = meas[2]
  error_func = TransformErrorEval(meas_T_21=T_k2_k1, T_20=T_k2_0, T_10=T_k1_0)
  cost_terms.append(WeightedLeastSquareCostTerm(error_func, noise_model, loss_func))

## Make Optimization Problem
problem = OptimizationProblem()
problem.add_state_var(*state_vars)
problem.add_cost_term(*cost_terms)

## Make solver and solve
solver = GaussNewtonSolver(problem, verbose=True, use_sparse_matrix=False)
solver.optimize()

Begin Optimization
------------------
Number of States:  99
Number of Cost Terms:  99
Initial Cost:  49.50495000000011
Iteration:    1  -  Cost:     0.0000
Iteration:    2  -  Cost:     0.0000
Termination Cause:  CONVERGED ABSOLUTE CHANGE
Total Optimization Time: 0.4793 seconds
